#### ROS 2 notes
1. *ros2 msg list* shows all message types
2. *ros2 msg show <msg>* shows the contents of the msg

In [ ]:
import os
import re
import csv
import sys
import json
import time
import pyfftw
import math
import matplotlib.pyplot as plt

from copy import deepcopy
from math import floor, copysign
from control.matlab import rlocus
from statistics import mean, stdev
from scipy import signal, optimize
from collections import OrderedDict
from control import mag2db, bode_plot, tf

import pandas as pd
import numpy as np

def sysidTfe(x, y, samplingFreq=1000, filterPhase=False):
    """ Performs transfer function estimation using spectral analysis methods. Similar to Matlab's etfe()

    Keyword arguments:
    x -- the array of input data
    y -- the array of output data
    samplingFreq -- the sampling frequency in Hz used to sample the data (default 1000)

    Keyword returns:
    db -- array of magnitude values representing the estimated transfer function
    phase -- array of phase values representing the estimated transfer function
    freqArr -- array of frequency values (in Hz) at which the transfer function is estimated
    """

    freqArr = np.arange(len(x))*samplingFreq/len(x)

    pyfftw.interfaces.cache.enable() # enable cache for extra speed
    x = pyfftw.byte_align(np.array(x)) # ensure arrays are aligned (for even more speed)
    y = pyfftw.byte_align(np.array(y))
    fftx = pyfftw.interfaces.numpy_fft.fft(x) # achieves results equivalent to np.fft.fft(x) but is much faster
    ffty = pyfftw.interfaces.numpy_fft.fft(y)
    fftyx = np.divide(ffty, fftx)

    phase = np.angle(fftyx, deg=True)
    mag = np.absolute(fftyx)
    db = mag2db(mag)

    if filterPhase:
        filterNumerator, filterDenominator = signal.butter(2, 0.03, btype='lowpass')
        phase = signal.filtfilt(filterNumerator, filterDenominator, phase)

    # correct for roll over 
    crossedZero = False
    crossed180 = False
    crossed360 = False
    for i in range(2, len(phase)):
        if phase[i-1] < -100:
            crossedZero = True
        if phase[i-1] < -250:
            crossed180 = True
        if phase[i-1] < -450:
            crossed360 = True
        if crossedZero and phase[i] > 0:
            phase[i] = phase[i] - 360
        if crossed180 and phase[i] > -180:
            phase[i] = phase[i] - 360
        if crossed360 and phase[i] > -360:
            phase[i] = phase[i] - 360

    return db, phase, freqArr

def sysidDelayEst(inputs, outputs, numPoles, numZeros, maxDelay=10):
    """ Finds the amount of pure time delay present in a system by applying ARX
    multiple times and returning the delay value which gives the smallest error.
    Similar to the Matlab function delayest()

    Keyword arguments:
    inputs -- the array of input data
    outputs -- the array of output data
    numPoles -- the number of poles you want to model the data with
    numZeros -- the number of zeros you want to model the data with
    maxDelay -- the maximum amount of delay you want to attempt to model the data with (default 15)

    Keyword returns:
    delay -- the estimate of the time delay of the system
    """
    print("sysid")
    
    errors = []
    for i in range(maxDelay):
        numerator, denominator, oneStepPrediction, meanErr, stdErr, normErr = sysid_arx(inputs, outputs, numPoles, numZeros, i)
        errors.append(normErr)
        
    print("sysid --")
    return np.argmin(errors)

def motorTorqueToSensedTorqueID(dataFilePath, figures=True, samplingFreq=1000, wpMax=100):
    """ Analyzes and reports on the 'torque plant ID' test data

    Keyword arguments:
    dataFilePath -- string containing the filepath of the csv file to analyze
    lowerGainCrossoverfrequency -- int containing gain crossover frequency lower bound
    upperGainCrossoverfrequency -- int containing gain crossover frequency upper bound 
    verbose -- if this boolean is true, extra info will be printed to the console
    figures -- if this boolean is true, figures will be displayed to the user

    Keyword returns:
    rawData -- OrderedDict containing the raw data contained in the csv file
    metrics -- dict containing the metrics associated with this test
    metricBounds -- dict containing the bounds for this test
    comment -- string commenting on the metrics
    """
    
    # parameters
    samplingTime = 1/samplingFreq;
    
    # loads raw data
    rawData = pd.read_csv(dataFilePath)
    if 'target_torque' not in rawData.columns:
        print('Error: "target_torque" is not a field in the given csv file. Check to make sure file is correct')
    if 'actual_torque' not in rawData.columns:
        print('Error: "actual_torque" is not a field in the given csv file. Check to make sure file is correct')
    motorTorque = rawData['target_torque'];
    sensedTorque = rawData['actual_torque']
        
    # zero out mean values
    inputs = np.subtract(motorTorque, mean(motorTorque))
    outputs = np.subtract(sensedTorque, mean(sensedTorque))
        
    # transfer function estimate with delay compensation
    numPoles = 2
    numZeros = 2 # 1
    delay = sysidDelayEst(inputs, outputs, numPoles, numZeros) # this takes about 15 seconds to compute. use as a debug option
    numerator, denominator, oneStepPrediction, meanErr, stdErr, normErr = sysid_arx(inputs, outputs, numPoles, numZeros, delay)
    sys = signal.TransferFunction(numerator, denominator, dt=samplingTime)
    w, mag, phase = signal.dbode(sys)
    w = w/(np.pi*2) # convert from rad/sec to hz
    gm, pm, wg, wp, wgInd, wpInd = calculateMargins(mag, phase, w)
    
    print("asdf")
        
    # transfer function estimate (only for frequencies below wpMax])
    if wp > wpMax:
        wp = float('nan')
    # add another delay if there is phase crossover frequency and identify model again 
    if wp > 0:
        delay = delay + 1
        numerator, denominator, oneStepPrediction, meanErr, stdErr, normErr = sysid_arx(inputs, outputs, numPoles, numZeros, delay)
        sys = signal.TransferFunction(numerator, denominator, dt=samplingTime)
        w, mag, phase = signal.dbode(sys)
        w = w/(np.pi*2) # convert from rad/sec to hz
        gm, pm, wg, wp, wgInd, wpInd = calculateMargins(mag, phase, w)    
    magEst, phaseEst, freqArr = sysidTfe(inputs, outputs)
    
    # this probably is wrong
    filterNumerator, filterDenominator = signal.butter(6, 0.005, btype='lowpass')
    magFilt = signal.filtfilt(filterNumerator, filterDenominator, magEst)
    phaseFilt = signal.filtfilt(filterNumerator, filterDenominator, phaseEst)

    if figures:
        print("----------------------------")
    
    if figures:
        print('Generating system ID figures. Please close plots after viewing to allow program to continue running')
        systf = tf(numerator, denominator, samplingTime)
        t = np.linspace(0, len(inputs)*samplingTime, num=len(inputs)) # generate time vector

        rlocus(systf)
        plt.title('Root Locus Plot of ARX Estimated System')

        plt.figure()
        plt.plot(t, rawData['target_current'], label='Current Input')
        plt.plot(t, rawData['actual_current'], label='Current Output')
        plt.grid()
        plt.legend(loc='upper right')
        plt.xlabel('Time (sec)')
        plt.ylabel('Current (Amps)')
        plt.title('Current Input vs Current Output')

        plt.figure()
        plt.plot(t, inputs, label='Torque Input')
        plt.plot(t, outputs, label='Torque Output')
        plt.grid()
        plt.legend(loc='upper right')
        plt.xlabel('Time (sec)')
        plt.ylabel('Torque (N*m)')
        plt.title('Torque Input vs Torque Output')

        plt.figure()
        plt.plot(t, outputs, label='Actual Output')
        plt.plot(t, oneStepPrediction, label='One Step Prediction')
        plt.grid()
        plt.legend(loc='upper right')
        plt.xlabel('Time (sec)')
        plt.ylabel('Torque (N*m)')
        plt.title('Actual Output vs One Step Prediction')

        plt.figure()
        minFreqInd = np.argmin(np.abs(w-1)) # find element in array closest to 1
        maxFreqInd = np.argmin(np.abs(w-100)) # find element in array closest to 100
        plt.semilogx(w[minFreqInd:maxFreqInd], mag[minFreqInd:maxFreqInd], label='ARX Estimation')    # Bode magnitude plot
        minFreqInd = np.argmin(np.abs(freqArr-1))
        maxFreqInd = np.argmin(np.abs(freqArr-100))
        plt.semilogx(freqArr[minFreqInd:maxFreqInd], magEst[minFreqInd:maxFreqInd], label='FFT Estimation')
        plt.semilogx(freqArr[minFreqInd:maxFreqInd], magFilt[minFreqInd:maxFreqInd], label='Filtered FFT')
        plt.semilogx(freqArr[minFreqInd:maxFreqInd], [0]*len(freqArr[minFreqInd:maxFreqInd]), linestyle='--', label='0 dB Line')
        if not np.isnan(wpInd):
            gainMargin = np.linspace(0, mag[wpInd], 10)
            plt.semilogx([w[wpInd]] * 10, gainMargin, color='k', linewidth=2)
        plt.legend(loc='lower left')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Magnitude (dB)')
        title = 'Gain margin: ' + '%.2f' % gm + ' dB with crossover frequency ' + '%.2f' % wp
        plt.title(title)
        plt.grid()

        plt.figure()
        minFreqInd = np.argmin(np.abs(w-1)) # find element in array closest to 1
        maxFreqInd = np.argmin(np.abs(w-100)) # find element in array closest to 100
        plt.semilogx(w[minFreqInd:maxFreqInd], phase[minFreqInd:maxFreqInd], label='ARX Estimation')  # Bode phase plot
        minFreqInd = np.argmin(np.abs(freqArr-1))
        maxFreqInd = np.argmin(np.abs(freqArr-100))
        plt.semilogx(freqArr[minFreqInd:maxFreqInd], phaseEst[minFreqInd:maxFreqInd], label='FFT Estimation')
        plt.semilogx(freqArr[minFreqInd:maxFreqInd], phaseFilt[minFreqInd:maxFreqInd], label='Filtered FFT')
        plt.semilogx(freqArr[minFreqInd:maxFreqInd], [-180]*len(freqArr[minFreqInd:maxFreqInd]), linestyle='--', label='-180 degree Line')
        if not np.isnan(wgInd):
            phaseMargin = np.linspace(-180, phase[wgInd], 10)
            plt.semilogx([w[wgInd]] * 10, phaseMargin, color='k', linewidth=2)
        plt.legend(loc='lower left')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Phase (deg)')
        title = 'Phase margin: ' + '%.2f' % pm + ' degrees with crossover frequency ' + '%.2f' % wg
        plt.title(title)
        plt.grid()
        plt.show()

motorTorqueToSensedTorqueID('/home/xyyeh/Desktop/torque-plant.csv')

In [ ]:
import os
import re
import csv
import sys
import json
import time
import pyfftw
import math
import matplotlib.pyplot as plt

from copy import deepcopy
from math import floor, copysign
from control.matlab import rlocus
from statistics import mean, stdev
from scipy import signal, optimize
from collections import OrderedDict
from control import mag2db, bode_plot, tf

import pandas as pd
import numpy as np


def sysid_arx(inputs, outputs, numPoles, numZeros, delay):
    """ Performs parametric system identification using the ARX method

    Keyword arguments:
    inputs -- the array of input data
    outputs -- the array of output data
    numPoles -- the number of poles you want to model the data with
    numZeros -- the number of zeros you want to model the data with
    delay -- the amount of delay you want to model the data with

    Keyword returns:
    tfNumerator -- the array representing the numerator of the estimated transfer function
    tfDenominator -- the array representing the denominator of the estimated transfer function
    oneStepPrediction -- the one step prediction using the estimated parameters
    meanError -- the mean of the prediction error. should be as close to zero as possible
    stdError -- the standard deviation of the prediction error. this should also be small
    normError -- the 2 norm of the prediction error. should be as small as possible
    """
    numZeros = numZeros + 1 # make the inputs and outputs indices match
    
    S = [[0] * (numPoles+numZeros)] * len(inputs)
    row = [0] * (numPoles+numZeros)
    
    # build the S matrix
    for k in range(numZeros+delay, len(inputs)):
        for i in range(0, numPoles):
            row[i] = -outputs[k-(i+1)]
        j = numZeros - 1
        for i in range(numPoles, numPoles+numZeros):
            row[i] = inputs[k-(numZeros+delay-j)]
            j = j - 1
        S[k] = deepcopy(row)

    # solve the overdetermined system to obtain our parameters
    theta = np.linalg.lstsq(S, outputs, -1)[0]
    
    # calculate prediction error metrics
    oneStepPrediction = np.matmul(S, theta)
    error = np.subtract(outputs, oneStepPrediction)
    meanError = mean(error)
    stdError = stdev(error)
    normError = np.linalg.norm(error)

    # build the numerator and denominator arrays
    tfNumerator = []
    tfDenominator = [1]
    for i in range(numPoles, numPoles+numZeros):
        tfNumerator.append(theta[i])
    for i in range(numPoles):
        tfDenominator.append(theta[i])
    # tfDenominator.extend([0] * delay)

    return tfNumerator, tfDenominator, oneStepPrediction, meanError, stdError, normError

def sysidTfe(x, y, samplingFreq=1000, filterPhase=False):
    """ Performs transfer function estimation using spectral analysis methods. Similar to Matlab's etfe()

    Keyword arguments:
    x -- the array of input data
    y -- the array of output data
    samplingFreq -- the sampling frequency in Hz used to sample the data (default 1000)

    Keyword returns:
    db -- array of magnitude values representing the estimated transfer function
    phase -- array of phase values representing the estimated transfer function
    freqArr -- array of frequency values (in Hz) at which the transfer function is estimated
    """

    freqArr = np.arange(len(x))*samplingFreq/len(x)

    pyfftw.interfaces.cache.enable() # enable cache for extra speed
    x = pyfftw.byte_align(np.array(x)) # ensure arrays are aligned (for even more speed)
    y = pyfftw.byte_align(np.array(y))
    fftx = pyfftw.interfaces.numpy_fft.fft(x) # achieves results equivalent to np.fft.fft(x) but is much faster
    ffty = pyfftw.interfaces.numpy_fft.fft(y)
    fftyx = np.divide(ffty, fftx)

    phase = np.angle(fftyx, deg=True)
    mag = np.absolute(fftyx)
    db = mag2db(mag)

    if filterPhase:
        filterNumerator, filterDenominator = signal.butter(2, 0.03, btype='lowpass')
        phase = signal.filtfilt(filterNumerator, filterDenominator, phase)

    # correct for roll over 
    crossedZero = False
    crossed180 = False
    crossed360 = False
    for i in range(2, len(phase)):
        if phase[i-1] < -100:
            crossedZero = True
        if phase[i-1] < -250:
            crossed180 = True
        if phase[i-1] < -450:
            crossed360 = True
        if crossedZero and phase[i] > 0:
            phase[i] = phase[i] - 360
        if crossed180 and phase[i] > -180:
            phase[i] = phase[i] - 360
        if crossed360 and phase[i] > -360:
            phase[i] = phase[i] - 360

    return db, phase, freqArr

def sysidDelayEst(inputs, outputs, numPoles, numZeros, maxDelay=10):
    """ Finds the amount of pure time delay present in a system by applying ARX
    multiple times and returning the delay value which gives the smallest error.
    Similar to the Matlab function delayest()

    Keyword arguments:
    inputs -- the array of input data
    outputs -- the array of output data
    numPoles -- the number of poles you want to model the data with
    numZeros -- the number of zeros you want to model the data with
    maxDelay -- the maximum amount of delay you want to attempt to model the data with (default 15)

    Keyword returns:
    delay -- the estimate of the time delay of the system
    """
    print("sysid")
    
    errors = []
    for i in range(maxDelay):
        numerator, denominator, oneStepPrediction, meanErr, stdErr, normErr = sysid_arx(inputs, outputs, numPoles, numZeros, i)
        errors.append(normErr)
        
    print("sysid --")
    return np.argmin(errors)

def motorTorqueToSensedTorqueID(dataFilePath, figures=True, samplingFreq=1000, wpMax=100):
    """ Analyzes and reports on the 'torque plant ID' test data

    Keyword arguments:
    dataFilePath -- string containing the filepath of the csv file to analyze
    lowerGainCrossoverfrequency -- int containing gain crossover frequency lower bound
    upperGainCrossoverfrequency -- int containing gain crossover frequency upper bound 
    verbose -- if this boolean is true, extra info will be printed to the console
    figures -- if this boolean is true, figures will be displayed to the user

    Keyword returns:
    rawData -- OrderedDict containing the raw data contained in the csv file
    metrics -- dict containing the metrics associated with this test
    metricBounds -- dict containing the bounds for this test
    comment -- string commenting on the metrics
    """
    
    # parameters
    samplingTime = 1/samplingFreq;
    
    # loads raw data
    rawData = pd.read_csv(dataFilePath)
    if 'target_torque' not in rawData.columns:
        print('Error: "target_torque" is not a field in the given csv file. Check to make sure file is correct')
    if 'actual_torque' not in rawData.columns:
        print('Error: "actual_torque" is not a field in the given csv file. Check to make sure file is correct')
    motorTorque = rawData['target_torque'];
    sensedTorque = rawData['actual_torque']
        
    # zero out mean values
    inputs = np.subtract(motorTorque, mean(motorTorque))
    outputs = np.subtract(sensedTorque, mean(sensedTorque))
        
    # transfer function estimate with delay compensation
    numPoles = 2
    numZeros = 2 # 1
    delay = sysidDelayEst(inputs, outputs, numPoles, numZeros) # this takes about 15 seconds to compute. use as a debug option
    numerator, denominator, oneStepPrediction, meanErr, stdErr, normErr = sysid_arx(inputs, outputs, numPoles, numZeros, delay)
    sys = signal.TransferFunction(numerator, denominator, dt=samplingTime)
    w, mag, phase = signal.dbode(sys)
    w = w/(np.pi*2) # convert from rad/sec to hz
    gm, pm, wg, wp, wgInd, wpInd = calculateMargins(mag, phase, w)
    
    print("asdf")
        
    # transfer function estimate (only for frequencies below wpMax])
    if wp > wpMax:
        wp = float('nan')
    # add another delay if there is phase crossover frequency and identify model again 
    if wp > 0:
        delay = delay + 1
        numerator, denominator, oneStepPrediction, meanErr, stdErr, normErr = sysid_arx(inputs, outputs, numPoles, numZeros, delay)
        sys = signal.TransferFunction(numerator, denominator, dt=samplingTime)
        w, mag, phase = signal.dbode(sys)
        w = w/(np.pi*2) # convert from rad/sec to hz
        gm, pm, wg, wp, wgInd, wpInd = calculateMargins(mag, phase, w)    
    magEst, phaseEst, freqArr = sysidTfe(inputs, outputs)
    
    # this probably is wrong
    filterNumerator, filterDenominator = signal.butter(6, 0.005, btype='lowpass')
    magFilt = signal.filtfilt(filterNumerator, filterDenominator, magEst)
    phaseFilt = signal.filtfilt(filterNumerator, filterDenominator, phaseEst)

    if figures:
        print("----------------------------")
    
    if figures:
        print('Generating system ID figures. Please close plots after viewing to allow program to continue running')
        systf = tf(numerator, denominator, samplingTime)
        t = np.linspace(0, len(inputs)*samplingTime, num=len(inputs)) # generate time vector

        rlocus(systf)
        plt.title('Root Locus Plot of ARX Estimated System')

        plt.figure()
        plt.plot(t, rawData['target_current'], label='Current Input')
        plt.plot(t, rawData['actual_current'], label='Current Output')
        plt.grid()
        plt.legend(loc='upper right')
        plt.xlabel('Time (sec)')
        plt.ylabel('Current (Amps)')
        plt.title('Current Input vs Current Output')

        plt.figure()
        plt.plot(t, inputs, label='Torque Input')
        plt.plot(t, outputs, label='Torque Output')
        plt.grid()
        plt.legend(loc='upper right')
        plt.xlabel('Time (sec)')
        plt.ylabel('Torque (N*m)')
        plt.title('Torque Input vs Torque Output')

        plt.figure()
        plt.plot(t, outputs, label='Actual Output')
        plt.plot(t, oneStepPrediction, label='One Step Prediction')
        plt.grid()
        plt.legend(loc='upper right')
        plt.xlabel('Time (sec)')
        plt.ylabel('Torque (N*m)')
        plt.title('Actual Output vs One Step Prediction')

        plt.figure()
        minFreqInd = np.argmin(np.abs(w-1)) # find element in array closest to 1
        maxFreqInd = np.argmin(np.abs(w-100)) # find element in array closest to 100
        plt.semilogx(w[minFreqInd:maxFreqInd], mag[minFreqInd:maxFreqInd], label='ARX Estimation')    # Bode magnitude plot
        minFreqInd = np.argmin(np.abs(freqArr-1))
        maxFreqInd = np.argmin(np.abs(freqArr-100))
        plt.semilogx(freqArr[minFreqInd:maxFreqInd], magEst[minFreqInd:maxFreqInd], label='FFT Estimation')
        plt.semilogx(freqArr[minFreqInd:maxFreqInd], magFilt[minFreqInd:maxFreqInd], label='Filtered FFT')
        plt.semilogx(freqArr[minFreqInd:maxFreqInd], [0]*len(freqArr[minFreqInd:maxFreqInd]), linestyle='--', label='0 dB Line')
        if not np.isnan(wpInd):
            gainMargin = np.linspace(0, mag[wpInd], 10)
            plt.semilogx([w[wpInd]] * 10, gainMargin, color='k', linewidth=2)
        plt.legend(loc='lower left')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Magnitude (dB)')
        title = 'Gain margin: ' + '%.2f' % gm + ' dB with crossover frequency ' + '%.2f' % wp
        plt.title(title)
        plt.grid()

        plt.figure()
        minFreqInd = np.argmin(np.abs(w-1)) # find element in array closest to 1
        maxFreqInd = np.argmin(np.abs(w-100)) # find element in array closest to 100
        plt.semilogx(w[minFreqInd:maxFreqInd], phase[minFreqInd:maxFreqInd], label='ARX Estimation')  # Bode phase plot
        minFreqInd = np.argmin(np.abs(freqArr-1))
        maxFreqInd = np.argmin(np.abs(freqArr-100))
        plt.semilogx(freqArr[minFreqInd:maxFreqInd], phaseEst[minFreqInd:maxFreqInd], label='FFT Estimation')
        plt.semilogx(freqArr[minFreqInd:maxFreqInd], phaseFilt[minFreqInd:maxFreqInd], label='Filtered FFT')
        plt.semilogx(freqArr[minFreqInd:maxFreqInd], [-180]*len(freqArr[minFreqInd:maxFreqInd]), linestyle='--', label='-180 degree Line')
        if not np.isnan(wgInd):
            phaseMargin = np.linspace(-180, phase[wgInd], 10)
            plt.semilogx([w[wgInd]] * 10, phaseMargin, color='k', linewidth=2)
        plt.legend(loc='lower left')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Phase (deg)')
        title = 'Phase margin: ' + '%.2f' % pm + ' degrees with crossover frequency ' + '%.2f' % wg
        plt.title(title)
        plt.grid()
        plt.show()

motorTorqueToSensedTorqueID('/home/xyyeh/Desktop/torque-plant.csv')
